In [6]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

# from manipulation.scenarios import SetColor

In [7]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


In [8]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the models to the plant.

# Transparent objects
# grasp1 = parser.AddModels("my_sdfs/wsg_transparent.sdf")[0]
# grasp2 = parser.AddModels("my_sdfs/wsg_transparent.sdf")[0]
# grasp3 = parser.AddModels("my_sdfs/wsg_transparent.sdf")[0]
# grasp4 = parser.AddModels("my_sdfs/wsg_transparent.sdf")[0]

grasp1 = parser.AddModels(file_name="../my_sdfs/wsg.sdf")[0]
grasp2 = parser.AddModels(file_name="../my_sdfs/wsg.sdf")[0]
grasp3 = 1#parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
grasp4 = 1#parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]

brick = parser.AddModels(file_name="../my_sdfs/prism.sdf")[0]
plant.Finalize()

models_list = [grasp1, grasp2, grasp3, grasp4]

In [9]:
B_O = plant.GetBodyByName("base_link", brick)
B_Ggrasp1 = plant.GetBodyByName("body", grasp1)
B_Ggrasp2 = plant.GetBodyByName("body", grasp2)
B_Ggrasp3 = 1#plant.GetBodyByName("body", grasp3)
B_Ggrasp4 = 1#plant.GetBodyByName("body", grasp4)

bodies_list = [B_O, B_Ggrasp1, B_Ggrasp2, B_Ggrasp3, B_Ggrasp4]

In [10]:

for body in bodies_list:
    if body == B_Ggrasp3 or body == B_Ggrasp4:
        continue
    if body == B_O:
        AddFrameTriadIllustration(
            plant=plant,
            scene_graph=scene_graph,
            body=body,
            length=0.09
        )
        continue
    AddFrameTriadIllustration(
        plant=plant,
        scene_graph=scene_graph,
        body=body,
        length=0.09
        # opacity=0.2,
    )

In [11]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [12]:

# Get the current object, O, pose
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

rotation_constraints = [0, np.pi/2, np.pi/2, np.pi]
p_GgraspO = [[0, 0.11 + 0.075, 0],
             [0, 0.11, 0.075],
             [0, 0.11, -0.075],
             [0, 0.11 + 0.075, 0]]

R_GgraspO = {}
X_GgraspO = {}
X_OGgrasp = {}
X_WGgrasp = {}

R_GgraspO[0] = RotationMatrix.MakeXRotation(rotation_constraints[0])
X_GgraspO[0] = RigidTransform(R_GgraspO[0], p_GgraspO[0])
X_OGgrasp[0] = X_GgraspO[0].inverse()
X_WGgrasp[0] = X_WO.multiply(X_OGgrasp[0])

R_GgraspO[1] = RotationMatrix.MakeXRotation(rotation_constraints[1])
X_GgraspO[1] = RigidTransform(R_GgraspO[1], p_GgraspO[1])
X_OGgrasp[1] = X_GgraspO[1].inverse()
X_WGgrasp[1] = X_WO.multiply(X_OGgrasp[1])

# R_GgraspO[2] = RotationMatrix.MakeXRotation(rotation_constraints[2])
# X_GgraspO[2] = RigidTransform(R_GgraspO[2], p_GgraspO[2])
# X_OGgrasp[2] = X_GgraspO[2].inverse()
# X_WGgrasp[2] = X_WO.multiply(X_OGgrasp[2])

# R_GgraspO[3] = RotationMatrix.MakeXRotation(rotation_constraints[3])
# X_GgraspO[3] = RigidTransform(R_GgraspO[3], p_GgraspO[3])
# X_OGgrasp[3] = X_GgraspO[3].inverse()
# X_WGgrasp[3] = X_WO.multiply(X_OGgrasp[3])


for i in range(len(rotation_constraints)):
    if i == 2 or i == 3:
        continue
    plant.SetFreeBodyPose(plant_context, bodies_list[i+1], X_WGgrasp[i])
    plant.GetJointByName("left_finger_sliding_joint", models_list[i]).set_translation(
        plant_context, -0.025
    )
    plant.GetJointByName("right_finger_sliding_joint", models_list[i]).set_translation(
        plant_context, 0.025
    )


In [13]:
diagram.ForcedPublish(context)